In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif
%matplotlib inline

In [ ]:
df = pd.read_sql('select * from sales', 
                 'sqlite:///../data/sales.db')

### Data Quality Check

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df = df.set_index('index')

In [ ]:
df.head()

### What coupon codes were used? How can I use them for predictive analytics?

In [ ]:
df['coupon_code'].value_counts()

In [ ]:
def used_coupon(code):
    if code == '':
        return 0
    else:
        return 1

In [ ]:
used_coupon('test')

In [ ]:
used_coupon('')

In [ ]:
df['coupon_code'].map(used_coupon).head()

In [ ]:
df['coupon_code'].map(lambda code: 0 if code == '' else 1).head()

In [ ]:
df['used_coupon'] = df['coupon_code'].map(
    lambda code: 0 if code == '' else 1)

In [ ]:
df.head()

### Adding customer data

In [ ]:
df = pd.read_sql('select * from sales join customers on sales.customer_id = customers."index"', 
                 'sqlite:///../data/sales.db')

In [ ]:
df.head()

### Cleaning it up - Be Specific!

In [ ]:
df = pd.read_sql('''select sales."index", coupon_code, branch_id, customer_id, 
                           sales.timestamp, sale_amount, email, city from sales 
                           join customers on sales.customer_id = customers."index"''', 
                 'sqlite:///../data/sales.db')

In [ ]:
df.head()

In [ ]:
df = df.set_index('index')

In [ ]:
df.head()

In [ ]:
df['used_coupon'] = df['coupon_code'].map(
    lambda code: 0 if code == '' else 1)

In [ ]:
df.head()

### Adding aggregate data

In [ ]:
df.groupby('customer_id')['sale_amount'].count()

In [ ]:
df['num_sales'] = df.groupby('customer_id')['sale_amount'].transform(
    'count')

In [ ]:
df.groupby('customer_id')['used_coupon'].transform('sum')

In [ ]:
df['num_sales_with_promo'] = df.groupby('customer_id')['used_coupon'].transform('sum')

In [ ]:
df['percent_coupon_use'] = df['num_sales_with_promo'] / df['num_sales']

In [ ]:
df.head()

### Looking at information gain

In [ ]:
df['percent_coupon_use'].hist()

In [ ]:
def get_likelihood(percentage):
    if percentage == 1:
        # 3: extremely likely (100%)
        return 3
    if percentage > .4:
        # 2: very likely (between 40-99%)
        return 2
    if percentage > 0:
        # 1: possibly (greater than 0, less than 40%)
        return 1
    # 0: unlikely (0%)
    return 0

In [ ]:
get_likelihood(1)

In [ ]:
get_likelihood(.3)

In [ ]:
df['coupon_likelihood'] = df['percent_coupon_use'].map(get_likelihood)

In [ ]:
df.head()

In [ ]:
y = df['coupon_likelihood']

In [ ]:
X = df.drop(['coupon_likelihood'], axis=1)

In [ ]:
mutual_info_classif(X.as_matrix(), y)

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])

In [ ]:
numeric_df.head()

In [ ]:
X = numeric_df.drop(['coupon_likelihood'], axis=1)

In [ ]:
mutual_info_classif(X.as_matrix(), y)

In [ ]:
mutual_info = mutual_info_classif(X.as_matrix(), y)

In [ ]:
for index, col in enumerate(X.columns):
    print('Feature: {}\n\tInformation Gain\t{}'.format(
        col, mutual_info[index]))

### Your Turn

- Are these valid features? Why or why not?